In [38]:
import cv2
import numpy as np
import os
import math as m
import re
import pandas as pd
import scipy
from scipy.stats import skew
from scipy.stats import kurtosis
import mahotas
from skimage.feature import local_binary_pattern
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [39]:
polygons = {}

In [40]:

def gauss(vertices):
    n = len(vertices)
    area = 0.0

    for i in range(n):
        x1, y1 = vertices[i]
        x2, y2 = vertices[(i + 1) % n]
        area += x1 * y2 - x2 * y1

    area = abs(area) / 2

    return area

def perimeter_calc(vertices):
    n = len(vertices)

    perimeter = 0.0

    for i in range(n):
        x1, y1 = vertices[i]
        x2, y2 = vertices[(i + 1) % n]
        perimeter += m.sqrt((x2 - x1)**2 + (y2 - y1)**2)
    return perimeter

def calculate_roundness(vertices):
    area = gauss(vertices)
    perimeter = perimeter_calc(vertices)

    roundness = 4 * m.pi * area / perimeter ** 2

    return "{:.4f}".format(roundness)

def compute_color_moments(vertices, img):

    vertices = np.array(vertices)
    polygon = vertices.reshape((-1,1,2)).astype(np.int32)

    #polygon = vertices.reshape((-1,1,2)).astype(np.int32)
    mask = np.zeros_like(img)
    cv2.fillPoly(mask, [polygon], (255,255,255))
    masked_image = cv2.bitwise_and(img, mask)
    gray_mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)

    average_color = cv2.mean(masked_image, mask=gray_mask)
    #BGRg
    #blue green red (gray/neutral)

    std_dev = np.std(masked_image)
    skewnessval = scipy.stats.skew(masked_image.flatten())
    kurt = kurtosis(masked_image.flatten())



    #print("Average color:", average_color)
    #print("std dev", std_dev)
    #print("skewness", skewnessval)
    #print("kurtosis", kurt)


    return average_color, std_dev, skewnessval, kurt
    ####

    '''
    
    imagepoly = img.copy()
    average_mask = img.copy()


    cv2.polylines(imagepoly, [polygon], True, (0, 255, 0), thickness=2)

    cv2.fillPoly(average_mask, [polygon], average_color)

    horizontal1 = np.hstack((img, imagepoly))
    horizontal2 = np.hstack((masked_image, average_mask))

    vertical = np.vstack((horizontal1, horizontal2))

    

    cv2.imshow("Image", vertical)



    cv2.waitKey(0)
    cv2.destroyAllWindows()

    '''



def compute_Zernike(vertices, img):
    vertices = np.array(vertices)
    polygon = vertices.reshape((-1,1,2)).astype(np.int32)
    mask = np.zeros_like(img)
    cv2.fillPoly(mask, [polygon], (255,255,255))
    
    masked_image = cv2.bitwise_and(img, mask)

    gray_mask = cv2.cvtColor(masked_image, cv2.COLOR_BGR2GRAY)

    zernike_moments = mahotas.features.zernike(gray_mask, radius=100, degree=8, cm=(100, 100))
    return zernike_moments.tolist()

def compute_LBP(vertices, img):
    #local binary patterns for texture
    vertices = np.array(vertices)
    polygon = vertices.reshape((-1,1,2)).astype(np.int32)
    mask = np.zeros_like(img)
    cv2.fillPoly(mask, [polygon], (255,255,255))
    
    masked_image = cv2.bitwise_and(img, mask)

    gray_mask = cv2.cvtColor(masked_image, cv2.COLOR_BGR2GRAY)

    lbp = local_binary_pattern(gray_mask, P=8, R=1, method='uniform')

    hist, _ = np.histogram(lbp, bins=np.arange(0, 11), density=True)

    #print("LBP Histogram:", hist)
    return hist.tolist()

def compute_HUMoments(vertices, img):
    vertices = np.array(vertices)
    polygon = vertices.reshape((-1,1,2)).astype(np.int32)
    mask = np.zeros_like(img)
    cv2.fillPoly(mask, [polygon], (255,255,255))
    
    masked_image = cv2.bitwise_and(img, mask)

    gray_mask = cv2.cvtColor(masked_image, cv2.COLOR_BGR2GRAY)

    moments = cv2.moments(gray_mask)

    hu_moments = cv2.HuMoments(moments).flatten()

    #print("Hu Moments:", hu_moments)
    for i in range(0,7):
        hu_moments[i] = -1* m.copysign(1.0, hu_moments[i]) * m.log10(abs(hu_moments[i]))

    
    return hu_moments.tolist()

    


# Retrieving features

This section receives the directory of the images and a directory containing the polygons that describe the form of each fruit

In [41]:
import cv2
import numpy as np
import os
import re
#"/Users/pablo/Desktop/Frutas/RaspsSSI.v1i.yolov8/train/images"
image_dir = "/Users/pablo/Desktop/Frutas/RaspsSSI.v1i.yolov8/train/images"
annotation_dir = "/Users/pablo/Desktop/Frutas/RaspsSSI.v1i.yolov8/train/labels"


regexpattern = r"^\d+"

for image_file in os.listdir(image_dir):
    #print(image_file)
    if image_file.endswith(".jpg"):

        image_path = os.path.join(image_dir, image_file)
        image = cv2.imread(image_path)
        #print(image.shape)
  

        annotation_file = os.path.splitext(image_file)[0] + ".txt"
        imageID = re.search(regexpattern, annotation_file).group(0)
        #print(imageID)
        #print(annotation_file)

        annotation_path = os.path.join(annotation_dir, annotation_file)
        #print(annotation_path)
        with open(annotation_path, "r") as f:
            annotations = f.readlines()
            #print(annotations)

        for annotation in annotations:
            # Split the annotation into its components
            components = annotation.split()
            class_id = int(components[0])
            coords = []

            for i in range(1, len(components)):
                corrected = 0
                if(i%2 == 0):
                    corrected = float(components[i]) * image.shape[0] 
                else:
                    corrected = float(components[i]) * image.shape[1]
                coords.append(corrected)

            points = np.array(coords).reshape((-1, 2)).astype(np.int32)
            print(points)
            #print(gauss(points))

            mean, stddev, skewnessval, kurt = compute_color_moments(points.tolist(), image)

            humoments = compute_HUMoments(points.tolist(), image)


            zernike = compute_Zernike(points.tolist(), image)

            localBinaryPatterns = compute_LBP(points.tolist(), image)
            
            #print(mean, stddev, skewnessval, kurt, humoments, zernike)
            polygons[imageID] = {"coordinates" : points.tolist(), 
                                 "area" : gauss(points), 
                                 "roundness" : calculate_roundness(points), 
                                 "meancolor" : mean,
                                 "meanB" : mean[0],
                                 "meanG" : mean[1],
                                 "meanR" : mean[2],
                                 "meangray" : mean[3],

                                 "stddev" : stddev,
                                 "skewness" : skewnessval,
                                 "kurtosis" : kurt,
                                 "h0" : humoments[0],
                                 "h1" : humoments[1],
                                 "h2" : humoments[2],
                                 "h3" : humoments[3],
                                 "h4" : humoments[4],
                                 "h5" : humoments[5],
                                 "h6" : humoments[6],
                                 "zernike" : zernike,
                                 "LBP" : localBinaryPatterns
                                 }

            #cv2.polylines(image, [points], True, (0, 255, 0), thickness=2)

        #horizontal = np.hstack((image, image))

        #cv2.imshow("Image", horizontal)
        #cv2.waitKey(0)
        #cv2.destroyAllWindows()

[[178 227]
 [180 235]
 [181 244]
 [187 263]
 [190 269]
 [195 273]
 [205 281]
 [217 287]
 [229 290]
 [240 296]
 [252 299]
 [261 303]
 [285 309]
 [292 309]
 [303 305]
 [316 298]
 [327 290]
 [335 279]
 [341 267]
 [343 262]
 [348 252]
 [351 242]
 [351 235]
 [348 230]
 [349 223]
 [346 214]
 [339 203]
 [338 197]
 [334 192]
 [332 185]
 [304 166]
 [279 155]
 [247 155]
 [238 154]
 [232 155]
 [226 157]
 [218 165]
 [212 166]
 [203 171]
 [190 187]
 [186 193]
 [185 201]
 [179 212]
 [178 220]
 [178 227]]
[[168 270]
 [178 286]
 [176 291]
 [176 295]
 [178 301]
 [180 307]
 [195 329]
 [201 339]
 [209 349]
 [218 361]
 [228 371]
 [232 374]
 [239 376]
 [254 375]
 [261 373]
 [267 372]
 [276 368]
 [284 366]
 [288 363]
 [297 352]
 [301 350]
 [305 346]
 [307 343]
 [309 334]
 [311 330]
 [320 332]
 [337 330]
 [343 328]
 [350 322]
 [354 314]
 [356 304]
 [356 300]
 [350 276]
 [346 269]
 [340 262]
 [315 243]
 [305 239]
 [286 218]
 [274 211]
 [262 203]
 [256 200]
 [249 200]
 [241 198]
 [231 198]
 [222 189]
 [215 186

In [42]:
IDAndArea = {}
IDAndCoordinates = {}
IDAndRoundness = {}

IDAndMeanColor = {}
IDAndMeanB = {}
IDAndMeanG = {}
IDAndMeanR = {}
IDAndMeanGray = {}
 
IDAndSTDDev = {}
IDAndSkewness = {}
IDAndKurtosis = {}

IDAndHUMoments = {}
IDAndHus = {}
IDAndH0 = {}
IDAndH1 = {}
IDAndH2 = {}
IDAndH3 = {}
IDAndH4 = {}
IDAndH5 = {}
IDAndH6 = {}

IDAndZernike = {}
IDAndLBP = {}

for elem in polygons:
    IDAndArea[elem] = polygons[elem]["area"]


    IDAndCoordinates[elem] = polygons[elem]["coordinates"]


    IDAndRoundness[elem] = polygons[elem]["roundness"]


    IDAndMeanColor[elem] = polygons[elem]["meancolor"]


    IDAndMeanB[elem] = polygons[elem]["meanB"]

    IDAndMeanG[elem] = polygons[elem]["meanG"]

    IDAndMeanR[elem] = polygons[elem]["meanR"]

    IDAndMeanGray[elem] = polygons[elem]["meangray"]


    IDAndSTDDev[elem] = polygons[elem]["stddev"]


    IDAndSkewness[elem] = polygons[elem]["skewness"]


    IDAndKurtosis[elem] = polygons[elem]["kurtosis"]



    IDAndH0[elem] = polygons[elem]['h0']

    IDAndH1[elem] = polygons[elem]['h1']

    IDAndH2[elem] = polygons[elem]['h2']

    IDAndH3[elem] = polygons[elem]['h3']

    IDAndH4[elem] = polygons[elem]['h4']

    IDAndH5[elem] = polygons[elem]['h5']

    IDAndH6[elem] = polygons[elem]['h6']



    IDAndZernike[elem] = polygons[elem]["zernike"]


    IDAndLBP[elem] = polygons[elem]["LBP"]

'''
print(IDAndArea)
print(IDAndCoordinates)
print(IDAndRoundness)
print(IDAndMeanColor)


print(IDAndSTDDev)
print(IDAndSkewness)
print(IDAndKurtosis)

print("h0",IDAndH0)
print("h1",IDAndH1)
print("h2",IDAndH2)
print("h3",IDAndH3)
print("h4",IDAndH4)
print("h5",IDAndH5)
print("h6",IDAndH6)

print("B",IDAndMeanB)
print("G",IDAndMeanG)
print("R",IDAndMeanR)
print("Gray",IDAndMeanGray)
'''
#print(IDAndZernike)
#print(IDAndLBP)


'\nprint(IDAndArea)\nprint(IDAndCoordinates)\nprint(IDAndRoundness)\nprint(IDAndMeanColor)\n\n\nprint(IDAndSTDDev)\nprint(IDAndSkewness)\nprint(IDAndKurtosis)\n\nprint("h0",IDAndH0)\nprint("h1",IDAndH1)\nprint("h2",IDAndH2)\nprint("h3",IDAndH3)\nprint("h4",IDAndH4)\nprint("h5",IDAndH5)\nprint("h6",IDAndH6)\n\nprint("B",IDAndMeanB)\nprint("G",IDAndMeanG)\nprint("R",IDAndMeanR)\nprint("Gray",IDAndMeanGray)\n'

# Area

In [43]:
areadf = pd.Series(IDAndArea).to_frame().reset_index()
areadf.columns = ['ID', 'AreaInSquaredPixels']
areadf['ID'] = areadf['ID'].astype(int)
areadf['AreaInSquaredPixels'] = areadf['AreaInSquaredPixels'].astype(float)


areadf = areadf.sort_values('ID')
areadf = areadf.reset_index(drop=True)

areadf.head(10)



,ID,AreaInSquaredPixels
0,1,19803.5
1,2,19730.5
2,3,19542.5
3,4,19550.0
4,5,19986.0
5,6,24299.5
6,7,24207.5
7,8,22393.0
8,9,24043.0
9,10,22789.0


# Roundness

In [44]:
roundnessdf = pd.Series(IDAndRoundness).to_frame().reset_index()
roundnessdf.columns = ['ID', 'Roundness']
roundnessdf['ID'] = roundnessdf['ID'].astype(int)
roundnessdf['Roundness'] = roundnessdf['Roundness'].astype(float)


roundnessdf = roundnessdf.sort_values('ID')
roundnessdf = roundnessdf.reset_index(drop=True)

roundnessdf.head(10)



,ID,Roundness
0,1,0.9648
1,2,0.9641
2,3,0.9617
3,4,0.9639
4,5,0.9701
5,6,0.8663
6,7,0.8921
7,8,0.8668
8,9,0.8922
9,10,0.8725


# Mass

In [45]:
def calculate_average_mass(df_row):
    sum_of_weights = 0

    for i in range(1,len(df_row)):
        sum_of_weights += float(df_row[i])
        average = sum_of_weights / (len(df_row) - 1)
    return "{:.4f}".format(average)

def rename_image(df_row):
    return df_row['Image'][8:]

def get_index(df_row):
    return df_row['Image'][:len(df_row['Image']) - 4]

In [46]:
weightsFileName = '/Users/pablo/Desktop/Frutas/weightsRaspberries/weightsRaspberries.txt'

df = pd.read_csv(weightsFileName, header=None, delim_whitespace=True)

df.rename(columns = {0:'Image'}, 
            inplace = True)

df['AverageMass'] = df.apply(calculate_average_mass, axis=1)

df['Image'] = df.apply(rename_image, axis=1)

df['ID'] = df.apply(get_index, axis=1)

mass = pd.DataFrame().assign(Image=df['Image'], ID=df['ID'], 
                            Weigh1=df[1],
                            Weigh2=df[2],
                            Weigh3=df[3],
                            Weigh4=df[4],
                            Weigh5=df[5],
                            Weigh6=df[6],
                            Weigh7=df[7],
                            Weigh8=df[8],
                            Weigh9=df[9],
                            Weigh10=df[10],
                            Weigh11=df[11],
                            AverageMass=df['AverageMass'])
mass['ID'] = mass['ID'].astype(int)
mass['AverageMass'] = mass['AverageMass'].astype(float)
mass.to_json('mass.json', orient = 'split', compression = 'infer', index = 'true')
mass.head(10)
latex_table = mass.head(10).to_latex(index=False, bold_rows=True, caption="Table of image Weighs and Average Mass")

#print(latex_table)


# Classic Geometry

In [47]:
classicgeometry = pd.merge(areadf[['ID', 'AreaInSquaredPixels']], mass[['ID', 'AverageMass']], on='ID', how='inner')

classicgeometry = pd.merge(classicgeometry[['ID', 'AverageMass', 'AreaInSquaredPixels']], roundnessdf[['ID', 'Roundness']], on='ID', how='inner')

classicgeometry.to_csv('classicgeometryRaspberriesRaw.csv', index = False)

classicgeometry.head(5)

#print(latex_table)

,ID,AverageMass,AreaInSquaredPixels,Roundness
0,1,4.8566,19803.5,0.9648
1,2,4.8568,19730.5,0.9641
2,3,4.8654,19542.5,0.9617
3,4,4.8691,19550.0,0.9639
4,5,4.8689,19986.0,0.9701


# Hu Moments

In [48]:
hu0 = pd.Series(IDAndH0).to_frame().reset_index()
hu0.columns = ['ID', 'hu0']
hu0['ID'] = hu0['ID'].astype(int)
hu0['hu0'] = hu0['hu0'].astype(float)
hu0 = hu0.sort_values('ID')
hu0 = hu0.reset_index(drop=True)

#print(hu0.head(10))

hu1 = pd.Series(IDAndH1).to_frame().reset_index()
hu1.columns = ['ID', 'hu1']
hu1['ID'] = hu1['ID'].astype(int)
hu1['hu1'] = hu1['hu1'].astype(float)
hu1 = hu1.sort_values('ID')
hu1 = hu1.reset_index(drop=True)

#print(hu1.head(10))

hu2 = pd.Series(IDAndH2).to_frame().reset_index()
hu2.columns = ['ID', 'hu2']
hu2['ID'] = hu2['ID'].astype(int)
hu2['hu2'] = hu2['hu2'].astype(float)
hu2 = hu2.sort_values('ID')
hu2 = hu2.reset_index(drop=True)

#print(hu2.head(10))

hu3 = pd.Series(IDAndH2).to_frame().reset_index()
hu3.columns = ['ID', 'hu3']
hu3['ID'] = hu3['ID'].astype(int)
hu3['hu3'] = hu3['hu3'].astype(float)
hu3 = hu3.sort_values('ID')
hu3 = hu3.reset_index(drop=True)

#print(hu3.head(10))

hu4 = pd.Series(IDAndH4).to_frame().reset_index()
hu4.columns = ['ID', 'hu4']
hu4['ID'] = hu4['ID'].astype(int)
hu4['hu4'] = hu4['hu4'].astype(float)
hu4 = hu4.sort_values('ID')
hu4 = hu4.reset_index(drop=True)

#print(hu4.head(10))


hu5 = pd.Series(IDAndH5).to_frame().reset_index()
hu5.columns = ['ID', 'hu5']
hu5['ID'] = hu5['ID'].astype(int)
hu5['hu5'] = hu5['hu5'].astype(float)
hu5 = hu5.sort_values('ID')
hu5 = hu5.reset_index(drop=True)

#print(hu5.head(10))


hu6 = pd.Series(IDAndH6).to_frame().reset_index()
hu6.columns = ['ID', 'hu6']
hu6['ID'] = hu6['ID'].astype(int)
hu6['hu6'] = hu6['hu6'].astype(float)
hu6 = hu6.sort_values('ID')
hu6 = hu6.reset_index(drop=True)

#print(hu6.head(10))
humoments = pd.merge(hu0[['ID', 'hu0']], mass[['ID', 'AverageMass']], on='ID', how='inner')

humoments = pd.merge(humoments[['ID', 'AverageMass', 'hu0']], hu1[['ID', 'hu1']], on='ID', how='inner')

humoments = pd.merge(humoments[['ID', 'AverageMass', 'hu0', 'hu1']], hu2[['ID', 'hu2']], on='ID', how='inner')
humoments = pd.merge(humoments[['ID', 'AverageMass', 'hu0', 'hu1', 'hu2']], hu3[['ID', 'hu3']], on='ID', how='inner')
humoments = pd.merge(humoments[['ID', 'AverageMass', 'hu0', 'hu1', 'hu2', 'hu3']], hu4[['ID', 'hu4']], on='ID', how='inner')
humoments = pd.merge(humoments[['ID', 'AverageMass', 'hu0', 'hu1', 'hu2', 'hu3', 'hu4']], hu5[['ID', 'hu5']], on='ID', how='inner')
humoments = pd.merge(humoments[['ID', 'AverageMass', 'hu0', 'hu1', 'hu2', 'hu3', 'hu4', 'hu5']], hu6[['ID', 'hu6']], on='ID', how='inner')


humoments.to_csv('humomentsRaspberriesRaw.csv', index = False)

humoments.head(5)
#
#0	1	2	3	4	5	6
#469	2.669828	6.950577	10.427229	10.260607	-20.705395	-13.776959	20.819507
#humoments.loc[468]

#print(latex_table)

,ID,AverageMass,hu0,hu1,hu2,hu3,hu4,hu5,hu6
0,1,4.8566,2.672802,6.999977,10.014052,10.014052,21.714844,15.067453,22.011631
1,2,4.8568,2.644730,6.961470,10.134569,10.134569,-20.846883,14.417799,-21.143387
2,3,4.8654,2.660988,6.888643,10.219095,10.219095,-21.806043,-14.812233,-20.933094
3,4,4.8691,2.678427,7.334345,10.110876,10.110876,21.927060,14.175830,20.804900
4,5,4.8689,2.658901,7.166956,9.886649,9.886649,-22.163944,-14.819426,-21.807110


# Color

In [49]:
meanB = pd.Series(IDAndMeanB).to_frame().reset_index()
meanB.columns = ['ID', 'meanB']
meanB['ID'] = meanB['ID'].astype(int)
meanB['meanB'] = meanB['meanB'].astype(float)
meanB = meanB.sort_values('ID')
meanB = meanB.reset_index(drop=True)

#print(meanB.head(10))

meanG = pd.Series(IDAndMeanG).to_frame().reset_index()
meanG.columns = ['ID', 'meanG']
meanG['ID'] = meanG['ID'].astype(int)
meanG['meanG'] = meanG['meanG'].astype(float)
meanG = meanG.sort_values('ID')
meanG = meanG.reset_index(drop=True)

#print(meanG.head(10))

meanR = pd.Series(IDAndMeanR).to_frame().reset_index()
meanR.columns = ['ID', 'meanR']
meanR['ID'] = meanR['ID'].astype(int)
meanR['meanR'] = meanR['meanR'].astype(float)
meanR = meanR.sort_values('ID')
meanR = meanR.reset_index(drop=True)

#print(meanR.head(10))

meanGray = pd.Series(IDAndMeanGray).to_frame().reset_index()
meanGray.columns = ['ID', 'meangray']
meanGray['ID'] = meanGray['ID'].astype(int)
meanGray['meangray'] = meanGray['meangray'].astype(float)
meanGray = meanGray.sort_values('ID')
meanGray = meanGray.reset_index(drop=True)

#print(meanGray.head(10))

colors = pd.merge(meanB[['ID', 'meanB']], mass[['ID', 'AverageMass']], on='ID', how='inner')

colors = pd.merge(colors[['ID', 'AverageMass','meanB']], meanG[['ID', 'meanG']], on='ID', how='inner')
colors = pd.merge(colors[['ID', 'AverageMass','meanB', 'meanG']], meanR[['ID', 'meanR']], on='ID', how='inner')
#colors = pd.merge(colors[['ID', 'AverageMass','meanB', 'meanG', 'meanR']], meanGray[['ID', 'meangray']], on='ID', how='inner')

colors.to_csv('colorsRaspberriesRaw.csv', index = False)


colors.head(5)

#print(latex_table)

,ID,AverageMass,meanB,meanG,meanR
0,1,4.8566,33.464530,48.613948,158.107883
1,2,4.8568,29.128493,42.534902,151.314171
2,3,4.8654,31.309586,45.554353,155.620589
3,4,4.8691,33.618352,48.424772,159.199393
4,5,4.8689,30.966370,46.179080,154.483333


# All data

In [50]:

allfeatures = pd.merge(classicgeometry[['ID', 'AverageMass', 'AreaInSquaredPixels', 'Roundness']], humoments[['ID', 'hu0', 'hu1', 'hu2', 'hu3', 'hu4', 'hu5', 'hu6']], on='ID', how='inner')
allfeatures = pd.merge(allfeatures[['ID', 'AverageMass', 'AreaInSquaredPixels', 'Roundness', 'hu0', 'hu1', 'hu2', 'hu3', 'hu4', 'hu5', 'hu6']], colors[['ID', 'meanB', 'meanG', 'meanR']], on='ID', how='inner')


allfeatures.to_csv('allfeaturesRaspberriesRaw.csv', index = False)

allfeatures.head(5)

,ID,AverageMass,AreaInSquaredPixels,Roundness,hu0,hu1,hu2,hu3,hu4,hu5,hu6,meanB,meanG,meanR
0,1,4.8566,19803.5,0.9648,2.672802,6.999977,10.014052,10.014052,21.714844,15.067453,22.011631,33.464530,48.613948,158.107883
1,2,4.8568,19730.5,0.9641,2.644730,6.961470,10.134569,10.134569,-20.846883,14.417799,-21.143387,29.128493,42.534902,151.314171
2,3,4.8654,19542.5,0.9617,2.660988,6.888643,10.219095,10.219095,-21.806043,-14.812233,-20.933094,31.309586,45.554353,155.620589
3,4,4.8691,19550.0,0.9639,2.678427,7.334345,10.110876,10.110876,21.927060,14.175830,20.804900,33.618352,48.424772,159.199393
4,5,4.8689,19986.0,0.9701,2.658901,7.166956,9.886649,9.886649,-22.163944,-14.819426,-21.807110,30.966370,46.179080,154.483333
